In [1]:
%cd /Users/gwg/dmcm
%matplotlib inline

/Users/gwg/dmcm


In [2]:
import matplotlib.pyplot as plt
import numpy as np
from   PIL import Image
import random
from   sklearn.externals import joblib
from   sklearn import preprocessing
import torch
from   torch.nn import functional as F

from   data import MnistConfig, MnistDataset
from   models import DPCCA, LeNet5AE, MAE, PCCA

In [3]:
cfg     = MnistConfig()
dataset = MnistDataset(cfg)

In [4]:
N  = 10000
x1 = torch.Tensor(N, cfg.N_CHANNELS, cfg.IMG_SIZE, cfg.IMG_SIZE)
x2 = torch.Tensor(N, cfg.N_GENES)
for i, (x1i, x2i) in enumerate(dataset):
    if i >= N:
        break
    x1[i] = x1i
    x2[i] = x2i

x = torch.cat([x1.view(N, 28*28), x2], dim=1)
print(x1.shape)
print(x2.shape)
print(x.shape)

torch.Size([10000, 1, 28, 28])
torch.Size([10000, 100])
torch.Size([10000, 884])


# Autoencoder

In [13]:
err1 = []
for seed in range(0, 5):
    path  = 'experiments/20190309_5x_ae_mnist_baseline/'\
            'mnist_mode-ae_seed-%s_latent_dim-2_lr-0.001_l1_coef-0.0_em_iters-1/'\
            'model.pt' % seed

    state = torch.load(path, map_location={'cuda:0': 'cpu'})
    model = LeNet5AE(cfg)
    model.load_state_dict(state)
    
    x1r = model.forward(x1)
    err1.append(F.mse_loss(x1r, x1).item())
    
print(round(np.mean(err1), 4))
print(round(np.std(err1), 4))

0.0196
0.0019


# MAE

In [14]:
err1 = []
err2 = []
for seed in range(0, 5):
    path  = '/Users/gwg/dmcm/experiments/20190307_5x_mae_mnist_baseline/'\
            'mnist_mode-mae_seed-%s_latent_dim-2_lr-0.001_l1_coef-0.0_em_iters-1/'\
            'model.pt' % seed

    state = torch.load(path, map_location={'cuda:0': 'cpu'})
    model = MAE(cfg)
    model.load_state_dict(state)
    
    x1r, x2r = model.forward([x1, x2])
    err1.append(F.mse_loss(x1r, x1).item())
    err2.append(F.mse_loss(x2r, x2).item())
    
print(round(np.mean(err1), 4))
print(round(np.std(err1), 4))
print(round(np.mean(err2), 4))
print(round(np.std(err2), 4))

0.0435
0.0015
2.287
0.0117


# PCCA

In [15]:
err1 = []
err2 = []
for seed in range(0, 5):
    path  = 'experiments/20190308_5x_pcca_mnist_baseline/'\
            'mnist_mode-pcca_seed-%s_latent_dim-2_lr-0.001_l1_coef-0.0_em_iters-1/model.pt' % seed

    state = torch.load(path, map_location={'cuda:0': 'cpu'})
    model = PCCA(latent_dim=2,
                 dims=[cfg.N_PIXELS, cfg.N_GENES],
                 gene_low_rank=cfg.GENE_EMBED_DIM,
                 differentiable=False)
    model.load_state_dict(state)

    z = model.estimate_z_given_y(x.t(), threshold=None)
    Lambda, Psi_diag = model.tile_params()
    xr = (Lambda @ z).t()

    x1r = xr[:, :cfg.N_PIXELS].view(-1, 1, 28, 28)
    x2r = xr[:, cfg.N_PIXELS:]

    err1.append(F.mse_loss(x1r, x1))
    err2.append(F.mse_loss(x2r, x2))
    
print(round(np.mean(err1), 4))
print(round(np.std(err1), 4))
print(round(np.mean(err2), 4))
print(round(np.std(err2), 4))

0.1207
0.0032
33.749
0.648


# DPCCA

In [16]:
err1 = []
err2 = []
for seed in range(0, 5):
    path  = '/Users/gwg/dmcm/experiments/20190307_5x_dpcca_mnist_baseline/'\
            'mnist_mode-dpcca_seed-%s_latent_dim-2_lr-0.001_l1_coef-0.0_em_iters-1/'\
            'model.pt' % seed

    state = torch.load(path, map_location={'cuda:0': 'cpu'})
    model = DPCCA(cfg, latent_dim=2)
    model.load_state_dict(state)
    
    x1r, x2r = model.forward([x1, x2])
    err1.append(F.mse_loss(x1r, x1).item())
    err2.append(F.mse_loss(x2r, x2).item())
    
print(round(np.mean(err1), 4))
print(round(np.std(err1), 4))
print(round(np.mean(err2), 4))
print(round(np.std(err2), 4))

0.0518
0.0121
2.3098
0.0137
